In [ ]:
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random


#NOTES:
## Håndtere i server som mottar join-koder at den enten sender trigger: riktig kode, eller trigger: feil kode, 
# utifra det så vet vi hvilken state vi skal til
#Hver eneste student må ha en slags kode/gruppenavn


class Student:
      def __init__(self):
            self.button_ask = widgets.Button(description="Ask")
            self.button_ask.on_click(self.on_button_ask)
            self.answered = ""
            self.message = ""
            self.name = "Herman"
            self.question = 0
            # text field
            self.text = widgets.Text(value='', placeholder='', description='String:', disabled=False)
            display(self.text, self.button_ask)
      
      def on_wait(self):
            """TODO"""
      
      def ask_help(self):
            question_for_server_use = self.name + " # " + self.question
            self.mqtt_client.publish(help, question_for_server_use)

      def finish_task(self):
           param = self.name + " $ " + self.question
           self.mqtt_client.publish('update', param)
           self.question += 1      






      def publish(self, message):
           self.mqtt_client.publish("join", message) 

              

      def on_button_join(self, session_code_stu):
        self.stm.send('join') # <---- here we send a message
        print(self.text.value)


# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions

#t1 and t2 are transitions from idle to wait and back
t1 = {'trigger':'join', 
      'source':'idle', 
      'target':'wait'}


t2 = {'trigger':'wrong_code', 
      'source':'wait', 
      'target':'idle'}


t3 = {'trigger':'correct_code', 
      'source':'wait', 
      'target':'lab_session_active'}



t4 = {'trigger':'ask_help', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'ask_help' }

t5 = {'trigger':'leave_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t6 = {'trigger':'finish_task', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t7 = {'trigger':'queue_request', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}


t8 = {'trigger':'leave_lab', 
      'source':'lab_session_active', 
      'target':'idle'}




# the states:
idle = {'name': 'idle'}

wait = {'name': 'wait',
        'entry': 'on_wait; publish(join, session_code)'}

lab_session_active = {'name': 'lab_session_active',}





class MQTT_Client_student:
    def __init__(self):
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        message = msg.payload.decode("utf-8")
        self.stm_driver.send(message, "student")

    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("")

        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()


# broker, port = 'iot.eclipse.org', 1883
broker, port = "localhost", 1883

student = Student()
student_interface = Machine(transitions=[t0, t1, t2], obj=student, name="student")
student.stm = student_interface

driver = Driver()
driver.add_machine(student_interface)

myclient = MQTT_Client_student()
student.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(broker, port)
    